In [1]:
from time import time

from tensorflow import keras
from PIL import Image
import numpy as np

YOUR_HDF5_MODEL = 'trees14-moblienetv2-200-acc0.714.h5'

model = keras.models.load_model(YOUR_HDF5_MODEL)

In [2]:
CAT = ['榕樹', '白千層', '楓香', '台灣欒樹', '小葉欖仁', '大葉欖仁', '茄冬',
       '黑板樹', '大王椰子', '鳳凰木', '阿勃勒', '水黃皮', '樟樹', '苦楝']

def preprocess_image(f, res=200):
    img = Image.open(f)
    img = img.resize((res, res))
    data = np.array(img.getdata())
    img = data.reshape(1, *img.size, 3)
    img = img.astype('float32')
    img /= 255.
            
    return img

In [6]:
now = time()
f = 'tm.jpg'
img = preprocess_image(f)
print(f'{CAT[np.argmax(model.predict(img))]} for {time()-now:.3f} secs')

小葉欖仁 for 0.112 secs


In [7]:
now = time()
fs = ['tm.jpg', 'bj.jpg', 'rr.jpg']
for f in fs:
    img = preprocess_image(f)
    print(f'{f}: {CAT[np.argmax(model.predict(img))]}')
print(f'for {time()-now:.3f} secs')

tm.jpg: 小葉欖仁
bj.jpg: 榕樹
rr.jpg: 大王椰子
for 0.276 secs


In [11]:
model.save('trees14-moblienetv2-200-acc0.714')

INFO:tensorflow:Assets written to: trees14-moblienetv2-200-acc0.714\assets


In [16]:
from tensorflow.experimental import tensorrt

params = tensorrt.ConversionParams(precision_mode='FP16')
converter = tensorrt.Converter(input_saved_model_dir="trees14-moblienetv2-200-acc0.714", conversion_params=params)
converter.convert()
converter.save(output_saved_model_dir='qq')

NotImplementedError: TensorRT integration is not available on Windows.

In [ ]:
tf.experimental.tensorrt.Converter(
    input_saved_model_dir='trees14-moblienetv2-200-acc0.714', input_saved_model_tags=None,
    input_saved_model_signature_key=None, use_dynamic_shape=None,
    dynamic_shape_profile_strategy=None, conversion_params=None
)

##  ONNX

In [1]:
import numpy as np
from PIL import Image

CAT = ['榕樹', '白千層', '楓香', '台灣欒樹', '小葉欖仁', '大葉欖仁', '茄冬',
       '黑板樹', '大王椰子', '鳳凰木', '阿勃勒', '水黃皮', '樟樹', '苦楝']

def preprocess_image(f, res=200):
    img = Image.open(f)
    img = img.resize((res, res))
    data = np.array(img.getdata())
    img = data.reshape(1, *img.size, 3)
    img = img.astype('float32')
    img /= 255.
            
    return img

In [15]:
import onnx
from onnx_tf.backend import prepare

img = preprocess_image('tm.jpg')
onnx_model = onnx.load("trees14-moblienetv2-200-acc0.714.onnx")  # load onnx model
r = prepare(onnx_model).run(img)  # run the loaded model
print(CAT[np.argmax(r)])

小葉欖仁


In [7]:
import onnxruntime
 
img = preprocess_image('rr.jpg')
session = onnxruntime.InferenceSession('trees14-moblienetv2-200-acc0.714.onnx')
r = session.run(None, {"input_8:0": img})
print(CAT[np.argmax(r)])

大王椰子


In [3]:
import netron
netron.start('trees14-moblienetv2-200-acc0.714.onnx')

Serving 'trees14-moblienetv2-200-acc0.714.onnx' at http://localhost:8080


('localhost', 8080)

## Channel First Model

In [1]:
from time import time

from tensorflow import keras
from PIL import Image
import numpy as np

YOUR_HDF5_MODEL = 'trees14-moblienetv3S400CF-ep156-loss1.914-acc0.580.h5'

model = keras.models.load_model(YOUR_HDF5_MODEL)

In [2]:
CAT = ['榕樹', '白千層', '楓香', '台灣欒樹', '小葉欖仁', '大葉欖仁', '茄冬',
       '黑板樹', '大王椰子', '鳳凰木', '阿勃勒', '水黃皮', '樟樹', '苦楝']

def preprocess_image(f, res=200):
    img = Image.open(f)
    img = img.resize((res, res))
    data = np.array(img.getdata())
    img = data.reshape(1, *img.size, 3)
    img = img.astype('float32')
    img = np.moveaxis(img, 3, 1)
            
    return img

In [5]:
now = time()
fs = ['tm.jpg', 'bj.jpg', 'rr.jpg']
for f in fs:
    img = preprocess_image(f, 400)
    print(f'{f}: {CAT[np.argmax(model.predict(img))]}')
print(f'for {time()-now:.3f} secs')

tm.jpg: 樟樹
bj.jpg: 茄冬
rr.jpg: 小葉欖仁
for 0.809 secs


In [6]:
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

now = time()
fs = ['tm.jpg', 'bj.jpg', 'rr.jpg']
for f in fs:
    img = preprocess_image(f, 400)
    print(f'{f}: {CAT[np.argmax(model.predict(img))]}')
print(f'for {time()-now:.3f} secs')

tm.jpg: 樟樹
bj.jpg: 茄冬
rr.jpg: 小葉欖仁
for 0.719 secs


In [10]:
# import onnx
# import onnxmltools

# keras_model = keras.models.load_model('trees14-moblienetv3S400CF-ep156-loss1.914-acc0.580.h5')
# onnx_path = 'some.onnx'
# onnx_model = onnxmltools.convert_keras(keras_model)
# onnx_model.graph.input[0].type.tensor_type.shape.dim[0].dim_value = 1
# onnx_model.graph.output[0].type.tensor_type.shape.dim[0].dim_value = 1
# onnx.checker.check_model(onnx_model)
# onnx.save(onnx_model, onnx_path)

ModuleNotFoundError: No module named 'onnxmltools'